In [ ]:
import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from isolated_ad_model.processing import PREPROCESS
from a2pm import A2PMethod

In [ ]:
class AdversarialAttack:
    def __init__(self):
        self.dataset = pd.read_csv('../isolated_ad_model/ue.csv')
        self.cols = self.dataset.columns

    @staticmethod
    def a2pm_attack(pattern, training_data, model):
        a2pm_method = A2PMethod(pattern)
        a2pm_method.fit(training_data)

        raw_adv_training_data = a2pm_method.generate(model, training_data)
        
        print(raw_adv_training_data)

        return pd.DataFrame(raw_adv_training_data, columns=training_data.columns)
    
    @staticmethod
    def predict(model, input):
        pred = model.predict(input)

        if -1 in pred:
            pred = [1 if p == -1 else 0 for p in pred]

            return pred
                
    @staticmethod
    def print_metrics(pred):
        print(f"Total number of inlier = {sum([p==0 for p in pred])}")
        print(f"Total number of outlier = {sum([p==1 for p in pred])}")

    def attack(self):
        true_anomalies = self.dataset['Viavi.UE.anomalies']
        print(self.dataset.columns)
        ps = PREPROCESS(self.dataset)  # TODO: Is it possible to get rid of src/scale dependency? 
        ps.process()
        training_data = ps.data
        # TODO: Original ADS does not use cross-validation. 

        # training_data.dropna()
        print(training_data.columns)

        iso = IsolationForest(n_estimators=100, contamination=0.25, max_features=1.0, random_state=42) # TODO: Import original ADS configuration as a baseline
        model = iso.fit(training_data)

        pred = self.predict(model, training_data)
        
        self.print_metrics(pred)

        f1_normal = f1_score(true_anomalies, pred)

        # TODO: Figure out patterns - how to optimize them
        pattern = (

                {
                    "type": "interval",
                    "features": None,
                    "ratio": 0.1,
                    "probability": 0.6,
                    "momentum": 0.99
                },
            )
            
        adv_training_data = self.a2pm_attack(pattern, training_data, model)

        print(training_data.columns)
        print(adv_training_data.columns)
        print(training_data.compare(adv_training_data))

        adv_pred = self.predict(model, adv_training_data)

        f1_adversarial = f1_score(true_anomalies, adv_pred)

        print("Regular f1_score ", f1_normal, ' vs. adversarial ', f1_adversarial, ' difference ', f1_normal- f1_adversarial)

        self.print_metrics(adv_pred)

        
   


In [29]:
attack = AdversarialAttack()
# attack.cols
attack.attack()

Index(['du-id', 'measTimeStampRf', 'nrCellIdentity', 'RRU.PrbUsedDl',
       'targetTput', 'DRB.UEThpDl', 'x', 'y', 'RF.serving.RSRP',
       'RF.serving.RSRQ', 'RF.serving.RSSINR', 'nbCellIdentity_0',
       'nbCellIdentity_1', 'nbCellIdentity_2', 'nbCellIdentity_3',
       'nbCellIdentity_4', 'rsrp_nb0', 'rsrq_nb0', 'rssinr_nb0', 'rsrp_nb1',
       'rsrq_nb1', 'rssinr_nb1', 'rsrp_nb2', 'rsrq_nb2', 'rssinr_nb2',
       'rsrp_nb3', 'rsrq_nb3', 'rssinr_nb3', 'rsrp_nb4', 'rsrq_nb4',
       'rssinr_nb4', 'Viavi.UE.anomalies', 'ue-id'],
      dtype='object')
Index(['du-id', 'RRU.PrbUsedDl', 'DRB.UEThpDl', 'x', 'y', 'RF.serving.RSRQ',
       'RF.serving.RSSINR'],
      dtype='object')
Total number of inlier = 7503
Total number of outlier = 2497


/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWar

[[ 4.40390672e-01  3.22737346e-02  1.11735082e-04 ... -5.13321023e-01
  -3.90675187e-03  1.94610034e-02]
 [ 6.97332635e-01  6.50410618e-02  8.55521999e-05 ... -7.21194147e-01
  -1.41680829e-02  2.28100895e-02]
 [ 4.68323567e-01  5.27071853e-02  5.54211954e-07 ... -7.80713545e-01
  -2.67461419e-02  3.16653411e-02]
 ...
 [ 9.10843217e-01  3.22737346e-02  1.37886294e-04 ...  4.55477939e-01
  -8.41866903e-03  1.17961310e-02]
 [ 4.70718854e-01  3.40981760e-02  1.89428448e-04 ... -2.65272943e-01
  -1.29907008e-02  2.12516179e-02]
 [ 6.38333683e-01  5.09922679e-02  4.14505720e-05 ... -3.86522630e-02
  -1.49303034e-02  1.89836501e-02]]
Index(['du-id', 'RRU.PrbUsedDl', 'DRB.UEThpDl', 'x', 'y', 'RF.serving.RSRQ',
       'RF.serving.RSSINR'],
      dtype='object')
Index(['du-id', 'RRU.PrbUsedDl', 'DRB.UEThpDl', 'x', 'y', 'RF.serving.RSRQ',
       'RF.serving.RSSINR'],
      dtype='object')
         du-id           RRU.PrbUsedDl           DRB.UEThpDl                \
          self     other      

/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/wysoc/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
